# Lab 3: Testing và Documentation - Data Quality trong dbt

## 🎯 Objectives
- Hiểu data quality testing trong dbt
- Sử dụng generic tests (unique, not_null, etc.)
- Tạo custom tests
- Viết documentation cho models và columns
- Generate và serve dbt documentation
- Sử dụng source freshness checks

## 📋 Prerequisites
- Hoàn thành Lab 1-2
- Models đã được run
- Hiểu về data quality concepts

## 🏗️ Testing Overview

**dbt Tests** đảm bảo data quality:
- Generic tests: Built-in tests (unique, not_null, etc.)
- Custom tests: SQL tests bạn tự viết
- Source freshness: Check data freshness từ sources
- Documentation: Describe models và columns

### Test Types:
```
┌─────────────┐
│ Generic Tests│  unique, not_null, relationships, etc.
└─────────────┘
       │
       ▼
┌─────────────┐
│ Custom Tests │  SQL tests trong tests/ directory
└─────────────┘
       │
       ▼
┌─────────────┐
│ Source Tests │  Freshness checks
└─────────────┘
```


## 1. Generic Tests

Generic tests là built-in tests trong dbt.


In [ ]:
# Xem schema.yml với tests
from pathlib import Path
import yaml

project_root = Path('.').resolve()
if project_root.name != 'dbt_lab':
    project_root = project_root.parent / 'dbt_lab'

schema_file = project_root / 'models' / 'marts' / 'schema.yml'

if schema_file.exists():
    print("📋 Schema với Tests:")
    print("=" * 60)
    with open(schema_file, 'r') as f:
        schema = yaml.safe_load(f)
        print(yaml.dump(schema, default_flow_style=False, sort_keys=False))
    print("=" * 60)
    
    print("\n💡 Generic Tests:")
    print("  - unique: Kiểm tra column có unique values không")
    print("  - not_null: Kiểm tra column không có NULL values")
    print("  - relationships: Kiểm tra foreign key relationships")
    print("  - accepted_values: Kiểm tra values trong allowed list")
    print("  - dbt_utils.*: Tests từ dbt_utils package")
else:
    print(f"⚠️  File not found: {schema_file}")


## 2. Common Generic Tests


In [ ]:
print("📊 Common Generic Tests:")
print("=" * 60)
print("""
1. unique
   - Kiểm tra column có unique values
   - Usage:
     columns:
       - name: customer_id
         tests:
           - unique

2. not_null
   - Kiểm tra không có NULL values
   - Usage:
     columns:
       - name: email
         tests:
           - not_null

3. relationships
   - Kiểm tra foreign key relationship
   - Usage:
     columns:
       - name: customer_id
         tests:
           - relationships:
               to: ref('stg_customers')
               field: customer_id

4. accepted_values
   - Kiểm tra values trong allowed list
   - Usage:
     columns:
       - name: status
         tests:
           - accepted_values:
               values: ['completed', 'pending', 'cancelled']

5. dbt_utils.expression_is_true
   - Kiểm tra SQL expression
   - Usage:
     columns:
       - name: lifetime_value
         tests:
           - dbt_utils.expression_is_true:
               expression: ">= 0"

6. dbt_utils.unique_combination_of_columns
   - Kiểm tra combination của columns là unique
   - Usage:
     tests:
       - dbt_utils.unique_combination_of_columns:
           combination_of_columns:
             - customer_id
             - order_date
""")
print("=" * 60)


## 3. Run Tests

Chạy tests với `dbt test`.


In [ ]:
# Run dbt tests
import subprocess
import os

def run_dbt_tests(select=None):
    """Run dbt tests"""
    print("🧪 Running dbt tests...")
    print("=" * 60)
    
    original_dir = os.getcwd()
    try:
        os.chdir(project_root)
        
        cmd = ['dbt', 'test', '--profiles-dir', '.', '--project-dir', '.']
        if select:
            cmd.extend(['--select', select])
        
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=60
        )
        
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ All tests passed!")
        else:
            print("\n❌ Some tests failed!")
            print("\nError output:")
            print(result.stderr)
            
    except Exception as e:
        print(f"⚠️  Error: {e}")
    finally:
        os.chdir(original_dir)

# Uncomment để chạy:
# run_dbt_tests()
# run_dbt_tests('customer_orders')  # Test một model cụ thể


## 4. Custom Tests

Custom tests là SQL tests bạn tự viết trong thư mục `tests/`.


In [ ]:
# Tạo example custom test
custom_test_example = """
-- tests/assert_positive_lifetime_value.sql
-- Custom test: Kiểm tra lifetime_value >= 0

select customer_id, lifetime_value
from {{ ref('customer_orders') }}
where lifetime_value < 0
"""

print("📝 Custom Test Example:")
print("=" * 60)
print(custom_test_example)
print("=" * 60)

print("\n💡 Custom Test Rules:")
print("  - File trong tests/ directory")
print("  - SQL query phải return 0 rows để pass")
print("  - Nếu return rows, test fails")
print("  - Có thể reference models với {{ ref() }}")
print("  - Có thể reference sources với {{ source() }}")

# Tạo file nếu chưa có
tests_dir = project_root / 'tests'
tests_dir.mkdir(exist_ok=True)

test_file = tests_dir / 'assert_positive_lifetime_value.sql'
if not test_file.exists():
    with open(test_file, 'w') as f:
        f.write(custom_test_example.strip())
    print(f"\n✅ Created: {test_file}")
else:
    print(f"\n✅ File already exists: {test_file}")


## 5. Documentation

Documentation giúp team hiểu models và columns.


In [ ]:
# Xem schema.yml với documentation
if schema_file.exists():
    print("📚 Documentation trong schema.yml:")
    print("=" * 60)
    with open(schema_file, 'r') as f:
        schema = yaml.safe_load(f)
    
    for model in schema.get('models', []):
        print(f"\nModel: {model.get('name', 'N/A')}")
        print(f"  Description: {model.get('description', 'N/A')}")
        print("\n  Columns:")
        for col in model.get('columns', []):
            print(f"    - {col.get('name', 'N/A')}: {col.get('description', 'No description')}")
    
    print("\n" + "=" * 60)
    
    print("\n💡 Documentation Best Practices:")
    print("  - Describe what model does")
    print("  - Explain business logic")
    print("  - Document all columns")
    print("  - Include examples khi cần")
    print("  - Link to related models")


## 6. Generate và Serve Documentation

Generate và serve dbt documentation để xem interactive docs.


In [ ]:
# Generate documentation
def generate_docs():
    """Generate dbt documentation"""
    print("📚 Generating dbt documentation...")
    print("=" * 60)
    
    original_dir = os.getcwd()
    try:
        os.chdir(project_root)
        
        result = subprocess.run(
            ['dbt', 'docs', 'generate', '--profiles-dir', '.', '--project-dir', '.'],
            capture_output=True,
            text=True,
            timeout=60
        )
        
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ Documentation generated successfully!")
            print("\n💡 To serve docs, run:")
            print("   dbt docs serve --profiles-dir . --project-dir .")
            print("   Then open http://localhost:8000")
        else:
            print("\n❌ Documentation generation failed!")
            print(result.stderr)
            
    except Exception as e:
        print(f"⚠️  Error: {e}")
    finally:
        os.chdir(original_dir)

# Uncomment để generate:
# generate_docs()


## 7. Source Freshness

Source freshness checks đảm bảo data sources được update đúng lúc.


In [ ]:
# Xem sources.yml với freshness
sources_file = project_root / 'models' / 'sources.yml'

if sources_file.exists():
    print("⏰ Source Freshness Configuration:")
    print("=" * 60)
    with open(sources_file, 'r') as f:
        sources = yaml.safe_load(f)
        print(yaml.dump(sources, default_flow_style=False, sort_keys=False))
    print("=" * 60)
    
    print("\n💡 Source Freshness:")
    print("  - warn_after: Warning nếu data stale")
    print("  - error_after: Error nếu data quá stale")
    print("  - loaded_at_field: Column chứa timestamp")
    print("\n  Usage:")
    print("    dbt source freshness --profiles-dir . --project-dir .")

# Run source freshness check
def check_source_freshness():
    """Check source freshness"""
    print("\n⏰ Checking source freshness...")
    print("=" * 60)
    
    original_dir = os.getcwd()
    try:
        os.chdir(project_root)
        
        result = subprocess.run(
            ['dbt', 'source', 'freshness', '--profiles-dir', '.', '--project-dir', '.'],
            capture_output=True,
            text=True,
            timeout=60
        )
        
        print(result.stdout)
        
        if result.returncode == 0:
            print("\n✅ Source freshness check completed!")
        else:
            print("\n⚠️  Source freshness check failed!")
            print(result.stderr)
            
    except Exception as e:
        print(f"⚠️  Error: {e}")
    finally:
        os.chdir(original_dir)

# Uncomment để check:
# check_source_freshness()


## 8. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. Generic tests (unique, not_null, relationships, etc.)
2. Custom tests trong tests/ directory
3. Documentation với YAML
4. Generate và serve documentation
5. Source freshness checks

### 📚 Next Lab:
- **Lab 4**: Macros và Jinja
- Tạo và sử dụng macros
- Jinja templating nâng cao
- Code reuse patterns

### 💡 Key Takeaways:

**Testing:**
- Generic tests: Built-in tests
- Custom tests: SQL tests bạn tự viết
- Tests chạy sau khi models run
- Tests fail nếu return rows

**Documentation:**
- YAML files mô tả models và columns
- Generate docs với `dbt docs generate`
- Serve docs với `dbt docs serve`
- Interactive lineage graphs

**Source Freshness:**
- Check data freshness từ sources
- warn_after và error_after thresholds
- Important cho data pipelines

### 🔗 Useful Links:
- [dbt Testing](https://docs.getdbt.com/docs/build/tests)
- [dbt Documentation](https://docs.getdbt.com/docs/collaborate/documentation)
- [dbt Source Freshness](https://docs.getdbt.com/docs/build/sources#snapshotting-source-data-freshness)
